<a href="https://colab.research.google.com/github/Jenoprabakar/TensorFlow-lab_experiments/blob/main/implementing_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Implement a Convolutional Neural Network (CNN) for image classification usingTensorFlow/Keras. (CIFAR-10 dataset).
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

# Load and normalize the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Reshape labels from 2D to 1D
y_train = y_train.reshape(-1,)
y_test = y_test.reshape(-1,)

# Define CNN architecture
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.Flatten(),

    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    x_train, y_train,
    epochs=10,
    validation_data=(x_test, y_test)
)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"\n Test Accuracy: {test_acc:.2f}")

# Plot training vs validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()
plt.show()


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 87s 54ms/step - accuracy: 0.3279 - loss: 1.8121 - val_accuracy: 0.5294 - val_loss: 1.3158
Epoch 2/10
 525/1563 ━━━━━━━━━━━━━━━━━━━━ 46s 44ms/step - accuracy: 0.5454 - loss: 1.2850

In [ ]:
#A healthcare company wants to build an automated X-ray image classification system to help radiologists identify whether a chest X-ray shows Normal or Pneumonia. You are asked to design and implement a Convolutional Neural Network (CNN) usingTensorFlow/Keras that preprocesses the dataset, trains the model, evaluates accuracy, and demonstrates predictions on sample test images. (Use Chest X-ray dataset from Kaggle).
# Install and import dependencies
!pip install -q kaggle

import os, random, math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from google.colab import files

# Upload kaggle.json to authenticate
uploaded = files.upload()

# Set up Kaggle API credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the chest X-ray pneumonia dataset
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia -q
!unzip -q chest-xray-pneumonia.zip -d chest_xray

# Define base directory
base_dir = '/content/chest_xray/chest_xray'

# Image preprocessing and data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (150, 150)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

validation_generator = val_test_datagen.flow_from_directory(
    os.path.join(base_dir, 'val'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    os.path.join(base_dir, 'test'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# Build CNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Set callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
    ModelCheckpoint('best_pneumonia_model.h5', monitor='val_loss', save_best_only=True)
]

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=callbacks
)

# Evaluate model performance
test_loss, test_acc = model.evaluate(test_generator)
print(f"\n Test loss: {test_loss:.4f} | Test accuracy: {test_acc:.4f}")

# Plot accuracy and loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend()
plt.title('Loss')
plt.show()

# Classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

test_generator.reset()
steps = math.ceil(test_generator.samples / test_generator.batch_size)
preds = model.predict(test_generator, steps=steps)

preds_class = (preds.ravel() >= 0.5).astype(int)
y_true = test_generator.classes
preds_class = preds_class[:len(y_true)]

print("\n📊 Classification Report:")
print(classification_report(y_true, preds_class, target_names=list(test_generator.class_indices.keys())))

cm = confusion_matrix(y_true, preds_class)
plt.figure(figsize=(5,4))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
plt.xticks([0,1], list(test_generator.class_indices.keys()), rotation=45)
plt.yticks([0,1], list(test_generator.class_indices.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Predict some random samples
from tensorflow.keras.preprocessing import image

filenames = test_generator.filepaths
sample_paths = random.sample(filenames, 6)

for p in sample_paths:
    img = image.load_img(p, target_size=IMG_SIZE)
    arr = image.img_to_array(img) / 255.0
    pred = model.predict(np.expand_dims(arr, axis=0))[0][0]
    label = 'PNEUMONIA' if pred >= 0.5 else 'NORMAL'

    plt.figure(figsize=(3,3))
    plt.imshow(img)
    plt.title(f"Pred: {label} ({pred:.2f})\nTrue: {os.path.basename(os.path.dirname(p))}")
    plt.axis('off')
    plt.show()
